In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd

# 1 SPARQL WikiData Query

In [ ]:


# Load your dataframe from the CSV file
df = pd.read_csv('activities_tc.csv')
df['city'] = df['city'].apply(lambda x: x.split(',')[0].strip())

# Define the SPARQL endpoint and the query
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

# Define the SPARQL query to fetch data about cities and municipalities in Spain
query = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX wikibase: <http://wikiba.se/ontology#>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>

# Retrieve distinct municipalities and cities in Spain
SELECT ?city ?cityLabel ?population ?loc WHERE {
  { 
    # Match cities in Spain
    ?city wdt:P31 wd:Q515 .         # Instance of a city
    ?city wdt:P17 wd:Q29 .          # Located in Spain
  } UNION {
    # Match municipalities in Spain
    ?city wdt:P31 wd:Q2074737 .     # Instance of a municipality
    ?city wdt:P17 wd:Q29 .          # Located in Spain
  }
  
  OPTIONAL { ?city wdt:P1082 ?population . }    # Get the population (optional)
  OPTIONAL { ?city wdt:P625 ?loc . }            # Get the location (optional)
  OPTIONAL { ?city rdfs:label ?cityLabel . }    # Get the label for the city/municipality
  
  FILTER (LANG(?cityLabel) = "es")              # Filter for Spanish labels
}
ORDER BY DESC(?population)
"""

# Set the query and request JSON results
sparql.setQuery(query)
sparql.setReturnFormat(JSON)

# Execute the query and fetch results
results = sparql.query().convert()

# Process the results into a list of dictionaries
cities_data = []
for result in results["results"]["bindings"]:
    city_info = {
        "city": result["city"]["value"],
        "cityLabel": result["cityLabel"]["value"] if "cityLabel" in result else None,
        "population": result["population"]["value"] if "population" in result else None,
        "loc": result["loc"]["value"] if "loc" in result else None
    }
    cities_data.append(city_info)

# Convert the list of dictionaries to a pandas DataFrame
df_sparql = pd.DataFrame(cities_data)
df['key'] = df.apply(lambda x: next((label for label in df_sparql['cityLabel'] if label in x['city']), None), axis=1)

# Merge using the 'key' column where the city contains the cityLabel
df_e = pd.merge(df, df_sparql, left_on="key", right_on="cityLabel", how="left")
# Print the number of matched rows (rows where there is non-null data from SPARQL results)
matched_rows = df_e[df_e['cityLabel'].notnull()]
unique_city_count = matched_rows['cityLabel'].nunique()
print(f"Number of matched cities: {unique_city_count}")

